In [1]:
import yfinance as yf
import pandas as pd
results= []
nifty50_tickers_ns = [
    "ADANIENT.ns",  # Adani Enterprises Ltd.
    "ADANIPORTS.ns",  # Adani Ports and SEZ Ltd.
    "APOLLOHOSP.ns",  # Apollo Hospitals Enterprise Ltd.
    "ASIANPAINT.ns",  # Asian Paints Ltd.
    "AXISBANK.ns",  # Axis Bank Ltd.
    "BAJFINANCE.ns",  # Bajaj Finance Ltd.
    "BAJAJFINSV.ns",  # Bajaj Finserv Ltd.
    "BHARTIARTL.ns",  # Bharti Airtel Ltd.
    "BIOCON.ns",  # Biocon Ltd.
    "BPCL.ns",  # Bharat Petroleum Corporation Ltd.
    "BRITANNIA.ns",  # Britannia Industries Ltd.
    "CIPLA.ns",  # Cipla Ltd.
    "COALINDIA.ns",  # Coal India Ltd.
    "DIVISLAB.ns",  # Divi's Laboratories Ltd.
    "DLF.ns",  # DLF Ltd.
    "DRREDDY.ns",  # Dr. Reddy's Laboratories Ltd.
    "EICHERMOT.ns",  # Eicher Motors Ltd.
    "GRASIM.ns",  # Grasim Industries Ltd.
    "HCLTECH.ns",  # HCL Technologies Ltd.
    # "HDFC.ns"
    "HDFCBANK.ns",  # HDFC Bank Ltd.
    "HDFCLIFE.ns",  # HDFC Life Insurance Company Ltd.
    "HEROMOTOCO.ns",  # Hero MotoCorp Ltd.
    "HINDALCO.ns",  # Hindalco Industries Ltd.
    "HINDUNILVR.ns",  # Hindustan Unilever Ltd.
    "ICICIBANK.ns",  # ICICI Bank Ltd.
    "ITC.ns",  # ITC Ltd.
    "JSWSTEEL.ns",  # JSW Steel Ltd.
    "KOTAKBANK.ns",  # Kotak Mahindra Bank Ltd.
    "LT.ns",  # Larsen & Toubro Ltd.
    "M&M.ns",  # Mahindra & Mahindra Ltd.
    "MARUTI.ns",  # Maruti Suzuki India Ltd.
    "NTPC.ns",  # NTPC Ltd.
    "ONGC.ns",  # Oil & Natural Gas Corporation Ltd.
    "POWERGRID.ns",  # Power Grid Corporation of India Ltd.
    "RELIANCE.ns",  # Reliance Industries Ltd.
    "SBILIFE.ns",  # SBI Life Insurance Company Ltd.
    "SBIN.ns",  # State Bank of India
    "SHREECEM.ns",  # Shree Cement Ltd.
    "SIEMENS.ns",  # Siemens Ltd.
    "SUNPHARMA.ns",  # Sun Pharmaceutical Industries Ltd.
    "TATAMOTORS.ns",  # Tata Motors Ltd.
    "TATAPOWER.ns",  # Tata Power Company Ltd.
    "TATASTEEL.ns",  # Tata Steel Ltd.
    "TECHM.ns",  # Tech Mahindra Ltd.
    "TITAN.ns",  # Titan Company Ltd.
    "ULTRACEMCO.ns",  # UltraTech Cement Ltd.
    "UPL.ns",  # UPL Ltd.
    "WIPRO.ns",  # Wipro Ltd.
    "YESBANK.ns",  # Yes Bank Ltd.
    "ZOMATO.ns"  # Zomato Ltd.
]
import os

In [3]:
# download data
if not os.path.exists(f"./database"):
            os.makedirs(f"./database")
for i in range(len(nifty50_tickers_ns)):
    ticker =   nifty50_tickers_ns[i]
    interval = '1h'  # Options: '1d', '1h', '5m', etc.
    period = '1mo'   # Duration to download, e.g., '1mo', '3mo', '1y', etc.

    # Download data
    data = yf.download(ticker, interval=interval, period=period)

    # Calculate 5 EMA and 20 EMA
    data['EMA_5'] = data['Close'].ewm(span=5, adjust=False).mean()
    data['EMA_20'] = data['Close'].ewm(span=20, adjust=False).mean()

    # Improved RSI Calculation
    def calculate_rsi(data, window=14):
        delta = data['Close'].diff()
        
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        
        avg_gain = gain.rolling(window=window, min_periods=1).mean()
        avg_loss = loss.rolling(window=window, min_periods=1).mean()
        
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        
        return rsi

    data['RSI'] = calculate_rsi(data)

    # Display the data with RSI and EMAs
    pd.DataFrame(data).dropna().to_csv(f"./database/{ticker}.csv")
    


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [45]:
def collade(firstart,firend,secstart,secend,type="default"):
    match type:
        case "AdowningB":
            if firstart>secstart and firend<secend:
                return collade(firstart,firend,secstart,secend)
        case "default":
            if firstart>firend:
                firstart,firend = firend , firstart
            if secstart>secend:
                secstart,secend = secend , secstart
            return [max(firstart, secstart), min(firend, secend)] if secstart <= firend and secend >= firstart else None

In [ ]:
for i in range(len(nifty50_tickers_ns)):
    ticker = nifty50_tickers_ns[i]
    data = pd.read_csv(f"./database/{ticker}.csv")
    data["EMA_5_PREV"] = data["EMA_5"].shift(1)    
    data["EMA_20_PREV"] = data["EMA_20"].shift(1) 
    data["RSI_PREV"] = data["RSI"].shift(1) 
    for j in range(data.shape[0]-data.shape[0],data.shape[0]):
        if data["RSI_PREV"].iloc[j]<data["RSI"].iloc[j] and data["RSI"].iloc[j]>=30 and data["RSI_PREV"].iloc[j]<30  and collade(data["EMA_5_PREV"].iloc[j],data["EMA_5"].iloc[j],data["EMA_20_PREV"].iloc[j],data["EMA_20"].iloc[j],"AdowningB")!=None:
            results.append([ticker,data["RSI"].iloc[j],data["RSI_PREV"].iloc[j],data["EMA_5"].iloc[j],data["EMA_5_PREV"].iloc[j],data["EMA_20"].iloc[j],data["EMA_20_PREV"].iloc[j]])
    pd.DataFrame(data,columns=["Datetime","Open","High","Low","Close","Adj Close","Volume","EMA_5","EMA_20","RSI","RSI_PREV","EMA_5_PREV","EMA_20_PREV"]).to_csv(f"./database/{ticker}.csv",index=False)
pd.DataFrame(results,columns=["Ticker","RSI","RSI_PREV","EMA_5","EMA_5_PREV","EMA_20","EMA_20_PREV"]).to_csv("testresults.csv",index=False)
    

26.81159653377766
27.814337965326487
28.37717093873904
29.21752417382808
29.894904733367696
26.45067761987393
25.22902956514197
27.897790126315243
27.67900557069292
26.33586015090784
27.270741028996795
24.56820053798849
29.763008735729173
26.962593192677573
27.40303258653134
29.19159542857977
23.38048768187205
28.801879610427108
29.26103563731528
18.10183556306906
26.958222046571507
22.65807725066584
26.302480327293097
29.038081592938937
26.4105161806729
26.81029647435897
29.83898595409387
22.08365579632792
28.877877869281832
20.26439824380165
29.929811714666315
28.352363788297367
27.38418224604061
27.345844504021443
23.31959508638127
25.212670309150013
24.3757148580154
28.740700016999483
29.38146924649992
26.3965656572545
28.49868610962132
23.529285047511905
24.91532682706812
19.73377217437435
24.956981227216858
24.161165465687432
29.782424268391964
28.66368850861538
29.488080154479903
27.039026223585463
24.24399187354443
29.55916714847811
24.55076703799557
29.76949498844027
26.531433